In [ ]:
%load_ext autoreload
import os, sys
sys.path.append(os.path.abspath('..'))

In [ ]:
import adaptive
import dependencies.adaptive_tools as adaptive_tools
adaptive.notebook_extension()
import holoviews as hv
from itertools import product
import matplotlib.pyplot as plt
import numpy as np
import cmath
import scipy.constants
from multiprocessing import Pool
from functools import partial
import kwant

import symmetry
import topology
import sns_system
import distributed_sns
import hpc05
import plotting_results
from copy import copy
import warnings

constants = dict(
    m_eff=0.023 * scipy.constants.m_e / (scipy.constants.eV * 1e-3) / 1e18,  # effective mass in kg, 
    hbar=scipy.constants.hbar / (scipy.constants.eV * 1e-3),
    current_unit=scipy.constants.k * scipy.constants.e / scipy.constants.hbar * 1e9,  # to get nA
    mu_B=scipy.constants.physical_constants['Bohr magneton'][0] / (scipy.constants.eV * 1e-3),
    exp=cmath.exp,
    cos=cmath.cos,
    sin=cmath.sin
   )

In [ ]:
client, dview, lview = hpc05.start_remote_and_connect(200, folder='~/two_dim_majoranas', timeout=1800)

In [ ]:
hpc05.kill_remote_ipcluster()

In [ ]:
try:
    runner.cancel()
except:
    pass

runner = adaptive.Runner(bl, executor=client)
bl.start_periodic_saver(runner,folder_path,interval=1800)

In [ ]:
runner.live_info()

In [ ]:
plotting_results.print_blearner_status(bl)

# Varying chemical potential and lattice spacing

In [ ]:
learners = []

mu = np.linspace(0,1.8,51)
a = list([5, 15.625])
transverse_soi = True
syst_pars = {'Ll' : 1000,
           'Lr' : 1000,
           'Lm' : 250,
           'Ly' : 4000,
           'a' :  15.625}

params = dict(g_factor = 10,
              mu = 1.8,
              alpha = 28,
              Delta = 0.18,
              B = 6,
              phase = 1)

folder_path = f'tsoi_{transverse_soi}_mu_{mu[-1]}[{len(mu)}]_a_{a}/'
folder_path = folder_path.replace(' ','_')
folder_path += '_'.join([f'{k}_{v}' for k, v in sorted({**params, **syst_pars}.items())])
# folder_path = f'tsoi_{transverse_soi}_mu_{mu}_a_{a}/'
# folder_path += '_'.join([f'{k}_{v}' for k, v in sorted({**params, **syst_pars}.items())])

def loss(ip):
    from adaptive.learner.learner2D import default_loss, areas
    adaptive.learner.learner2D.np
    loss = default_loss(ip)
    dim = areas(ip).shape[0]
    return 1e8 * loss / dim if dim < 2000 else loss    

for _mu,_a in product(mu, a):
    params['mu'] = _mu
    syst_pars['a'] = _a
    f = partial(distributed_sns.f_adaptive, keys=('B', 'phase'), params=copy(params), 
            syst_pars=copy(syst_pars), transverse_soi=transverse_soi)
    learner = adaptive_tools.Learner2D(f, bounds=[(0, 1.2), (-1.0*np.pi, 1.0*np.pi)], loss_per_triangle=loss)
    
    learner.pars = (_mu,_a)
    learners.append(learner)


bl = adaptive_tools.BalancingLearner(learners)

try:
    bl.load(folder_path)
except:
    pass

In [ ]:
plt_dict = {learner.pars: learner.plot(n=200, tri_alpha=0.3).redim(x='B (T)', y = 'Phase')
            for learner in bl.learners}
int_plot = hv.plotting.plot.HoloMap(plt_dict, kdims=['Wn'])

int_plot

In [ ]:
plotter = lambda l: l.plot(tri_alpha=0.0).redim(x='B (T)', y = 'Phase')
int_plot = bl.plot(cdims=(['mu', 'a'], product(mu, a)), plotter=plotter)

In [ ]:
renderer = hv.renderer('bokeh').instance(fig='auto', holomap='widgets')
renderer.save(int_plot, 'phase_zeeman', style=dict(Image={'cmap':'viridis', 'trialpha' : 0}))

In [ ]:
from IPython.display import display_html
res = display_html(renderer.(int_plot), raw=True)

In [ ]:
int_plot

# Varying width of superconductor

In [ ]:
learners = []

# mu = np.linspace(0,1.8,4)
WSC = np.linspace(500,1500,5)
transverse_soi = True
syst_pars = {'Ll' : 1000,
           'Lr' : 1000,
           'Lm' : 250,
           'Ly' : 50,
           'a' :  15.625}

params = dict(g_factor = 10,
              mu = .18,
              alpha = 28,
              Delta = 0.18,
              B = 0.2,
              phase = 1*np.pi)

folder_path = f'tsoi_{transverse_soi}_WSC_{WSC}/'
folder_path += '_'.join([f'{k}_{v}' for k, v in sorted({**params, **syst_pars}.items())])

def loss(ip):
    from adaptive.learner.learner2D import default_loss, areas
    adaptive.learner.learner2D.np
    loss = default_loss(ip)
    dim = areas(ip).shape[0]
    return 1e8 * loss / dim if dim < 2000 else loss    

for _WSC in WSC:
    syst_pars['Ll'] = _WSC
    syst_pars['Lr'] = _WSC
    
    f = partial(distributed_sns.f_adaptive, keys=('B', 'phase'), params=copy(params), 
            syst_pars=copy(syst_pars), transverse_soi=transverse_soi)
    learner = adaptive_tools.Learner2D(f, bounds=[(0, 1.2), (-1.0*np.pi, 1.0*np.pi)], loss_per_triangle=loss)
    
    learner.pars = (_WSC)
    learners.append(learner)


bl = adaptive_tools.BalancingLearner(learners)

try:
    bl.load(folder_path)
except:
    pass

# Varying the superconductor width
We expect to see no difference if we change the size of the superconductors, as they should be 'large enough' to have no influence on the result. We indeed see than for the first 'diamond', the difference is negligible. For larger magnetic fields, the results diverge amongst each other, but it is debatable whether this region is physical in any case.

In [ ]:
plt_dict = {learner.pars: learner.plot(n=200, tri_alpha=0.3).redim(x='B (T)', y = 'Phase')
            for learner in bl.learners}
int_plot = hv.plotting.plot.HoloMap(plt_dict, kdims=['WSC'])

int_plot

# Varying channel width
Here we vary the width of the normal region, which should shift the thouless energy:
\begin{equation}
E_T = \frac{\pi}{2} \frac{\hbar v_f}{W} 
\end{equation}

In [ ]:
try:
    runner.cancel()
except:
    pass


runner = adaptive.Runner(bl, executor=client)
runner.live_info()

bl.start_periodic_saver(runner,folder_path,interval=1800)

In [ ]:
learners = []

# mu = np.linspace(0,1.8,4)
Wn = [250, 500, 1000, 2000,5000] #np.linspace(250, 2500, 5)
transverse_soi = True
syst_pars = {'Ll' : 1000,
           'Lr' : 1000,
           'Lm' : 250,
           'Ly' : 50,
           'a' :  25}

params = dict(g_factor = 10,
              mu = .18,
              alpha = 28,
              Delta = 0.18,
              B = 0.2,
              
              phase = 1*np.pi)

folder_path = f'tsoi_{transverse_soi}_Wn_{Wn}/'
folder_path += '_'.join([f'{k}_{v}' for k, v in sorted({**params, **syst_pars}.items())])

def loss(ip):
    from adaptive.learner.learner2D import default_loss, areas
    adaptive.learner.learner2D.np
    loss = default_loss(ip)
    dim = areas(ip).shape[0]
    return 1e8 * loss / dim if dim < 2000 else loss    

for _Wn in Wn:
    syst_pars['Lm'] = _Wn
    
    f = partial(distributed_sns.f_adaptive, keys=('B', 'phase'), params=copy(params), 
            syst_pars=copy(syst_pars), transverse_soi=transverse_soi)
    learner = adaptive_tools.Learner2D(f, bounds=[(0, .4), (-1.0*np.pi, 1.0*np.pi)], loss_per_triangle=loss)
    
    learner.pars = tuple([_Wn])
    learners.append(learner)


bl = adaptive_tools.BalancingLearner(learners)

try:
    pass
#     bl.load(folder_path.replace('  ',' '))
except:
    pass

In [ ]:
3**2*(np.pi**2 * constants['hbar']**2 / (2*constants['m_eff'] * 2000**2))

In [ ]:
plt_dict = {learner.pars: learner.plot(n=200).redim(x='B (T)', y = 'Phase')
            for learner in bl.learners}


int_plot = hv.plotting.plot.HoloMap(plt_dict, kdims=['Wn'])


int_plot


In [ ]:
(134*delta(5000))/(2*10*constants['mu_B'])

In [ ]:
template = sns_system.get_template_strings(transverse_soi=True)
ham = kwant.continuum.discretize(template, grid_spacing = syst_pars['a'])



In [ ]:
b_to_Ez(0.024) / delta(5000)

In [ ]:
def b_to_Ez(B):
    return 2*B*params['g_factor']*constants['mu_B']

def delta(W):
    return (constants['hbar']**2*np.pi**2/(2*constants['m_eff']*(2*W)**2))


In [ ]:
2*.056*constants['mu_B']*10/(constants['hbar']**2*np.pi**2/(2*constants['m_eff']*(2*2000)**2))

In [ ]:
2*.175*constants['mu_B']*10/(constants['hbar']**2*np.pi**2/(2*constants['m_eff']*(2*2000)**2))


In [ ]:
def make_sns_system(a, Lm, Lr, Ll, Ly, transverse_soi = True):
    """ 
    Builds and returns finalized 2dim sns system
    
    Parameters:
    -----------
    a : float
        lattice spacing in nm

    Lm : float
        width of middle normal strip

    Lr : float
        width of right superconductor

    Ll : float
        width of left superconductor

    Ly : float
        length of finite system

    Returns:
    --------
    syst : kwant.system.FiniteSystem
        Finite system where lead[0] is assumed to be the bulk lead, a slice of the bulk along the y-axis
    """

    #     HAMILTONIAN DEFINITIONS
    template_strings = sns_system.get_template_strings(transverse_soi)

    # TURN HAMILTONIAN STRINGS INTO TEMPLATES
    template_normal = kwant.continuum.discretize(template_strings['ham_normal'], grid_spacing=a)
    template_sc_left = kwant.continuum.discretize(template_strings['ham_sc_left'], grid_spacing=a)
    template_sc_right = kwant.continuum.discretize(template_strings['ham_sc_right'], grid_spacing=a)
    
    # SHAPE FUNCTIONS
    def shape_normal(site):
        (x, y) = site.pos
        return 0 <= y < Ly and 0 <= x < Lm
    
    def shape_left_sc(site):
        (x, y) = site.pos
        return 0 <= y < Ly and -Ll <= x < 0
    
    def shape_right_sc(site):
        (x, y) = site.pos
        return 0 <= y < Ly and Lm <= x < Lm + Lr
    def shape_lead(x1, x2):
        def shape(site):
            (x, y) = site.pos
            return  x1 <= x < x2
        return shape
    
    # BUILD FINITE SYSTEM
    syst = kwant.Builder()

    syst.fill(template_normal, shape_normal, (0,0))
    syst.fill(template_sc_left, shape_left_sc, (-Ll,0))
    syst.fill(template_sc_right, shape_right_sc, (Lm,0))

    # LEAD: SLICE OF BULK ALONG Y AXIS
    lead = kwant.Builder(kwant.TranslationalSymmetry([0,-a]))
    
    lead.fill(template_sc_left, shape_lead(-Ll,0), (-Ll, -a))
    lead.fill(template_normal, shape_lead(0,Lm), (0,-a))
    lead.fill(template_sc_right, shape_lead(Lm,Lm + Lr), (Lm,-a))

    syst.attach_lead(lead)
    syst.attach_lead(lead.reversed())
    
    return syst.finalized()

In [ ]:
syst_pars = {'Ll' : 1000,
           'Lr' : 1000,
           'Lm' : 250,
           'Ly' : 25,
           'a' :  25}

params = dict(**constants,
              g_factor = 10,
              mu = .18,
              alpha = 28,
              Delta = 0.18,
              B = 0.4,
              phase = 1*np.pi)

syst = make_sns_system(**syst_pars)


In [ ]:
smat = kwant.smatrix(syst, energy=0.18, params=params)

In [ ]:
smat.num_propagating(0)

In [ ]:
16*(constants['hbar']**2*np.pi**2/(2*constants['m_eff']*(2*250)**2))/(2*10*constants['mu_B'])


In [ ]:
try:
    runner.cancel()
except:
    pass


runner = adaptive.Runner(bl, executor=client)
runner.live_info()

bl.start_periodic_saver(runner, folder_path,interval=1800)

In [ ]:
kf = np.sqrt(2*360*constants['m_eff'])/constants['hbar']
vf = constants['hbar']*kf/constants['m_eff'] #+ params['alpha']/constants['hbar']
print(vf*1e-9/1e5)

In [ ]:
def thouless_in_tesla(W):
    
#     kf = np.sqrt(2*params['mu']*constants['m_eff'])/constants['hbar']
#     vf = constants['hbar']*kf/constants['m_eff'] + params['alpha']/constants['hbar']
#     E_T = np.pi/2*constants['hbar']*vf/W
    E_T = np.pi**2 * constants['hbar']**2 / (2 * constants['m_eff'] *(2*W)**2)
    return E_T/2/params['g_factor']/constants['mu_B']
plt_dict = {learner.pars: (learner.plot(n=100, tri_alpha=0.3).redim(x='B (T)', y = 'Phase').relabel(
            f'B_T = {thouless_in_tesla(learner.pars[0]):.4}'))
            for learner in bl.learners}
int_plot = hv.plotting.plot.HoloMap(plt_dict, kdims=['Wn'])

int_plot

# Thouless calculation

In [ ]:
%autoreload

In [ ]:
def calcTransmission(energy):
    import cmath
    import numpy as np
    import kwant
    import scipy.constants
    import sns_system
    
    constants = dict(
    m_eff=0.023 * scipy.constants.m_e / (scipy.constants.eV * 1e-3) / 1e18,  # effective mass in kg, 
    hbar=scipy.constants.hbar / (scipy.constants.eV * 1e-3),
    current_unit=scipy.constants.k * scipy.constants.e / scipy.constants.hbar * 1e9,  # to get nA
    mu_B=scipy.constants.physical_constants['Bohr magneton'][0] / (scipy.constants.eV * 1e-3),
    exp=cmath.exp,
    cos=cmath.cos,
    sin=cmath.sin
   )
    syst_pars = {'Ll' : 1000,
               'Lr' : 1000,
               'Lm' : 250,
               'Ly' : 4000,
               'a' :  50}

#     params = dict(**constants,
#                   g_factor = 10,
#                   mu = .18,
#                   alpha = 28,
#                   Delta = 0.18,
#                   B = 0.4,
#                   phase = 1*np.pi)
    params = dict(**constants,
                  g_factor = 10,
                  mu = .18,
                  alpha = 28,
                  Delta = 0.18,
                  B = 0.4,
                  phase = energy*np.pi)
    
    syst = sns_system.make_junction(**syst_pars)
    
    smat = kwant.smatrix(syst, params = params, energy = .36)
    return smat.transmission(1, 0)

learner = adaptive.Learner1D(calcTransmission, bounds=(-4,4))

In [ ]:
try:
    runner.cancel()
except:
    pass

runner = adaptive.Runner(learner, executor=client)
runner.live_info()

In [ ]:
learner.plot()

In [ ]:
smat.conductance_matrix()